# Backtesting MutantBaby

## Backtest Method
    30 days as a group, randomly choose 30 continuos 30 days to see the performance

In [1]:
# from __future__ import (absolute_import, division, print_function,
#                         unicode_literals)
import datetime
# import os.path
import sys

import pandas as pd
import numpy as np
import backtrader as bt
import backtrader.indicators as ta

import matplotlib
import tkinter
matplotlib.use('TKAgg')
# matplotlib.use('QT5Agg')

from mutant.model import MutantBaby
from mutant.strategy import MutantBacktrader

raw_data_path = "../data/BTCUSD_latest.csv"
# raw_data_path = "../data/Raw_BTCUSDT1708-2303.csv"

## Load data

In [2]:
dataframe = pd.read_csv(raw_data_path,
                                parse_dates=True,
                                index_col=0)
dataframe.index = pd.to_datetime(dataframe.index, format='ISO8601')

In [3]:
dataframe

,open,high,low,close,volume
date,,,,,
2017-08-17 04:00:00,4261.48,4261.48,4261.48,4261.48,7564.90
2017-08-17 04:01:00,4261.48,4261.48,4261.48,4261.48,0.00
2017-08-17 04:02:00,4280.56,4280.56,4280.56,4280.56,1117.54
2017-08-17 04:03:00,4261.48,4261.48,4261.48,4261.48,51.17
2017-08-17 04:04:00,4261.48,4261.48,4261.48,4261.48,599.99
...,...,...,...,...,...
2023-07-04 21:15:00,30794.96,30794.96,30790.20,30790.21,202250.26
2023-07-04 21:16:00,30790.21,30791.08,30790.20,30791.08,210922.27
2023-07-04 21:17:00,30791.08,30791.08,30780.94,30780.95,407579.78


In [4]:
load_params = np.load("out.npz", allow_pickle=True)

In [5]:
load_params.keys()

KeysView(NpzFile 'out.npz' with keys: populations, objective, control_params, op_objective)

In [6]:
load_params = np.load("out-op_control_param.npz", allow_pickle=True)
control_params = {'tp': [0.6], 'sl': [1.8]}
for key in load_params:
    control_params[key] = load_params[key]

control_params

{'tp': array([1.28352904]),
 'sl': array([0.75406508]),
 'ema_1_length': array([34]),
 'ema_2_length': array([11]),
 'ema_3_length': array([30]),
 'macd_fast_length': array([33]),
 'macd_slow_length': array([44]),
 'macd_signal_length': array([47]),
 'rsi_length': array([38]),
 'rsi_long': array([66]),
 'rsi_short': array([42])}

In [7]:
# control_params = {
#     'tp': np.array([1.84]),
#     'sl': np.array([1.23]),
#     'ema_1_length': np.array([41]),
#     'ema_2_length': np.array([28]),
#     'ema_3_length': np.array([82]),
#     'macd_fast_length': np.array([31]),
#     'macd_slow_length': np.array([42]),
#     'macd_signal_length': np.array([22]),
#     'rsi_length': np.array([43]),
#     'rsi_long': np.array([44]),
#     'rsi_short': np.array([47])
# }

In [8]:
model = Mutant()
print(model.params)
model.update_params(control_params)
print(model.params)

{'tp': [0.6], 'sl': [1.8], 'ema_1_length': [126], 'ema_2_length': [156], 'ema_3_length': [121], 'macd_fast_length': [45], 'macd_slow_length': [54], 'macd_signal_length': [45], 'rsi_length': [14], 'rsi_long': [51], 'rsi_short': [48]}
{'tp': array([1.28352904]), 'sl': array([0.75406508]), 'ema_1_length': array([34]), 'ema_2_length': array([11]), 'ema_3_length': array([30]), 'macd_fast_length': array([33]), 'macd_slow_length': array([44]), 'macd_signal_length': array([47]), 'rsi_length': array([38]), 'rsi_long': array([66]), 'rsi_short': array([42])}


In [9]:
print("TP:", model.tp, ", SL:", model.sl)

TP: 1.2835290378577748 , SL: 0.754065077877389


In [10]:
trade_reports = []
sharp_reports = []
drawdown_reports = []
total_sessions = 24
for i in range(total_sessions):
    backtest_length = 1440*30
    start = np.random.choice(len(dataframe) - backtest_length)
    end = start + backtest_length
    df = dataframe.iloc[start:end]
    df = df.groupby(pd.Grouper(freq='5Min')).agg({"open": "first", 
                                                  "high": "max",
                                                  "low": "min",
                                                  "close": "last",
                                                  "volume": "sum"})
    data = bt.feeds.PandasData(dataname=df, datetime=None,)
    cerebro = bt.Cerebro()
    cerebro.addstrategy(MutantBacktrader, model, print_log=False)
    cerebro.adddata(data)
    cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='mutant_trade')
    cerebro.addanalyzer(bt.analyzers.DrawDown, _name='mutant_drawdown')
    cerebro.addanalyzer(
        bt.analyzers.SharpeRatio,
        timeframe=bt.TimeFrame.Days, 
        compression=1, 
        factor=365,
        annualize =True,
        _name='mutant_sharpe'
    )
    cerebro.addsizer(bt.sizers.PercentSizer, percents=99)
    cerebro.broker.setcash(100000.0)
    cerebro.broker.setcommission(commission=0.0004)
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
    results = cerebro.run()
    result = results[0]
    trade_reports.append(result.analyzers.mutant_trade.get_analysis())
    sharp_reports.append(result.analyzers.mutant_sharpe.get_analysis())
    drawdown_reports.append(result.analyzers.mutant_drawdown.get_analysis())
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 100000.00
Final Portfolio Value: 100000.00
Starting Portfolio Value: 100000.00
Final Portfolio Value: 100000.00
Starting Portfolio Value: 100000.00
Final Portfolio Value: 100000.00
Starting Portfolio Value: 100000.00
Final Portfolio Value: 100000.00
Starting Portfolio Value: 100000.00
Final Portfolio Value: 100000.00
Starting Portfolio Value: 100000.00
Final Portfolio Value: 100000.00
Starting Portfolio Value: 100000.00
Final Portfolio Value: 100000.00
Starting Portfolio Value: 100000.00
Final Portfolio Value: 100000.00
Starting Portfolio Value: 100000.00
Final Portfolio Value: 100000.00
Starting Portfolio Value: 100000.00
Final Portfolio Value: 100000.00
Starting Portfolio Value: 100000.00
Final Portfolio Value: 99377.68
Starting Portfolio Value: 100000.00
Final Portfolio Value: 100000.00
Starting Portfolio Value: 100000.00
Final Portfolio Value: 100000.00
Starting Portfolio Value: 100000.00
Final Portfolio Value: 100000.00
Starting Portfolio Value: 100000.00

In [11]:
sharp_reports

[OrderedDict([('sharperatio', None)]),
 OrderedDict([('sharperatio', None)]),
 OrderedDict([('sharperatio', None)]),
 OrderedDict([('sharperatio', None)]),
 OrderedDict([('sharperatio', None)]),
 OrderedDict([('sharperatio', None)]),
 OrderedDict([('sharperatio', None)]),
 OrderedDict([('sharperatio', None)]),
 OrderedDict([('sharperatio', None)]),
 OrderedDict([('sharperatio', None)]),
 OrderedDict([('sharperatio', -3.9617507633730114)]),
 OrderedDict([('sharperatio', None)]),
 OrderedDict([('sharperatio', None)]),
 OrderedDict([('sharperatio', None)]),
 OrderedDict([('sharperatio', None)]),
 OrderedDict([('sharperatio', nan)]),
 OrderedDict([('sharperatio', None)]),
 OrderedDict([('sharperatio', None)]),
 OrderedDict([('sharperatio', None)]),
 OrderedDict([('sharperatio', None)]),
 OrderedDict([('sharperatio', None)]),
 OrderedDict([('sharperatio', None)]),
 OrderedDict([('sharperatio', None)]),
 OrderedDict([('sharperatio', None)])]

In [12]:
""" Trade report is provided by Backtrader.analyzers.TradeAnalyzer

"""
def get_win_rate(trade_raport):
    """ Get the win rate from trade report
    
    Win rate = number of win trades / number of total trades
    """
    total_trades = trade_raport['total']['total']
    win_trades = trade_raport['won']['total']
    win_rate = win_trades / total_trades
    return win_rate

def get_roi(trade_raport, init_protfolio_value=100000.0):
    """ Get the ROI from trade report

    ROI = PNL / Initial Portfolio Value
    """
    pnl = trade_raport['pnl']['net']['total']
    roi = pnl / init_protfolio_value
    return roi

def get_drawdown(drawdown_report):
    """ Get the drawdown from drawdown report

    """
    drawdown = drawdown_report['drawdown']
    return drawdown

In [13]:
sessions_win = 0
sessions_win_rate = 0
roi_s = []
for i in range(total_sessions):
    trade_report = trade_reports[i]
    if trade_report['total']['total'] > 0:
        drawdown_report = drawdown_reports[i]
        win_rate = get_win_rate(trade_report)
        roi = get_roi(trade_report)
        roi_s.append(roi)
        if roi > 0:
            sessions_win += 1
        drawdown = get_drawdown(drawdown_report)
        print("total_trades: {}, win_rate: {:.2f},  roi: {:.2f}%, drawdown: {:.2f}%.".format(trade_report['total']['total'], win_rate, roi*100, drawdown))
    else:
        print("No trade.")
        
sessions_win_rate = sessions_win/total_sessions * 100
roi_avg = sum(roi_s) / len(roi_s)
print("sessions_win_rate: {:.2f}%, roi_avg: {:.4f}%.".format(sessions_win_rate, roi_avg)) 

No trade.
No trade.
No trade.
No trade.
No trade.
No trade.
No trade.
No trade.
No trade.
No trade.
total_trades: 1, win_rate: 0.00,  roi: -0.62%, drawdown: 0.62%.
No trade.
No trade.
No trade.
No trade.
No trade.
No trade.
No trade.
No trade.
No trade.
No trade.
No trade.
No trade.
No trade.
sessions_win_rate: 0.00%, roi_avg: -0.0062%.
